In [ ]:
import requests
from PIL import Image, ImageDraw
from io import BytesIO
import base64

# Authentication tokens and URLs for the APIs
LLAVA_NEXT_URL = "https://supplement-convenient-scores-forgotten.trycloudflare.com/llava/generate"
LLAVA_NEXT_TOKEN = "Mklm9M70UL39aQhBLiDSDM2yv1V73uXWMA-GBN1LBDU"
FLUX_URL = "https://maintained-thai-filter-four.trycloudflare.com/imagine/generate"
FLUX_TOKEN = "calm-bold-tree-bd1b1c8bc141403231e7f0a8"
SDXL_URL = "https://singing-shadows-duty-loan.trycloudflare.com/imagine/generate"
SDXL_TOKEN = "wise-radiant-wave-b77c84b64e9efa2ebf8710ca"

def summarize_story(context):
    # Define the token and endpoint
    VALID_TOKEN = "AEulafWZQ1xhIkeLgjcp5nhL7d0cLPWiXAU34DTVvXI"
    url = "https://cu-vertical-dimensional-continuity.trycloudflare.com/phi3/generate"
    # Set headers
    headers = {
        "Authorization": f"Bearer {VALID_TOKEN}",
        "Content-Type": "application/json"
    }
    # Define payload
    payload = {
        "inputs": "<|system|>\nYou are a creative writing assistant who can summarize large sentences in few words. Summarize the given prompt in less than 200 characters.<|end|>\n<|user|>\n Prompt is: "+context+".<|end|>\n<|assistant|>",
        "parameters": {
            "max_new_tokens": 300,
            "temperature": 0.7
        }
    }
    # Make the request
    response = requests.post(url, headers=headers, json=payload)
    # Display the response
    if response.status_code == 200:
        print(response.json().get("generated_text", "No response text available"))
    #else:
    #    ; #print(f"Error: {response.status_code}, {response.text}")
    summary=response.json().get("generated_text") #{response.text}
    return summary

# Function to evaluate if the initial prompt is clear enough
def evaluate_prompt(prompt):
    if len(prompt.split()) < 10:
        return False  # Not enough detail
    return True  # Assuming it's detailed enough for now

# Function to enhance the prompt if it's not detailed enough
def enhance_prompt(prompt):
    headers = {
        "Authorization": f"Bearer {LLAVA_NEXT_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "inputs": f"<|system|>\nYou are a creative manga story writer.<|end|>\n<|user|>\nThe following prompt is vague. Please enhance the prompt to be detailed enough for an entire story. Prompt: {prompt}\n<|assistant|>",
        "parameters": {"max_new_tokens": 150}
    }
    response = requests.post(LLAVA_NEXT_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text", "").strip()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return prompt  # If enhancement fails, return original prompt

# Function to convert image to base64 encoding
def image_to_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

# Function to generate story sentence from a prompt using LLava-Next API
def generate_story_sentence_llava(prompt, previous_sentence="", previous_image_path="", max_tokens=500):
    headers = {
        "Authorization": f"Bearer {LLAVA_NEXT_TOKEN}",
        "Content-Type": "application/json"
    }

    # Convert previous image to base64
    print("previous_image_path-> ",  previous_image_path)
    previous_image_base64 = image_to_base64(previous_image_path) if previous_image_path else ""

    # Prepare the prompt and the image for the LLava-Next model
    input_data = {
        "inputs": f"Story Context: {prompt}. \n Previous sentence: {previous_sentence}",
        "parameters": {
            "max_new_tokens": max_tokens,  # Limit output to 500 tokens
            "temperature": 0.7,
            "top_p": 0.9,
        },
        "image": previous_image_base64,  # Include the base64 encoded previous image for style context
        "instruction": "Use the Story Context to maintain global consistency in story creation. Generate next sentence given the Previous sentence, and the previous image. give only one sence"
    }

    response = requests.post(LLAVA_NEXT_URL, headers=headers, json=input_data)

    if response.status_code == 200:
        #print (response.json().get("generated_text", "").strip())
        return response.json().get("generated_text", "").strip()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return ""

# Function to generate story sentence from a prompt using LLava-Next API
def generate_story_sentence_only_text_llava(prompt, previous_sentence="", previous_image_path="", max_tokens=500):
    headers = {
        "Authorization": f"Bearer {LLAVA_NEXT_TOKEN}",
        "Content-Type": "application/json"
    }

    # Convert previous image to base64
    #print("previous_image_path " +  previous_image_path)
    previous_image_base64 = image_to_base64(previous_image_path) if previous_image_path else ""

    # Prepare the prompt and the image for the LLava-Next model
    input_data = {
        "inputs": f"Story Context: {prompt}. \n Previous sentence: {previous_sentence}",
        "parameters": {
            "max_new_tokens": max_tokens,  # Limit output to 500 tokens
            "temperature": 0.7,
            "top_p": 0.9,
        },
        "instruction": "Use the Story Context to maintain global consistency in story creation. Generate next sentence given the prompt and Previous sentence."
    }

    response = requests.post(LLAVA_NEXT_URL, headers=headers, json=input_data)

    if response.status_code == 200:
        #print (response.json().get("generated_text", "").strip())
        return response.json().get("generated_text", "").strip()

    else:
        print ("o no")
        print(f"Error: {response.status_code}, {response.text}")
        return ""

# Function to generate image from text using Flux API
def generate_image(description, previous_image_description, output_file):
    headers = {
        "Authorization": f"Bearer {FLUX_TOKEN}",
        "Content-Type": "application/json"
    }
    # Combine sentence description with the description of the previous image for context
    combined_description = f"{description}. Previous Image: {previous_image_description}"
    payload = {
        "prompt": combined_description[:200],  # Max 200 characters for prompt
        "img_size": 512,
        "guidance_scale": 7.5,
        "num_inference_steps": 50,
        "seed":  101,
    }
    response = requests.post(FLUX_URL, headers=headers, json=payload)
    if response.status_code == 200:
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"Image saved to {output_file}")
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Function to generate image using SDXL API for higher resolution
def generate_sdxl_image(description, previous_image_description, output_file):
    headers = {
        "Authorization": f"Bearer {SDXL_TOKEN}",
        "Content-Type": "application/json"
    }
    # Combine sentence description with the description of the previous image for context
    combined_description = f"{description}. #Previous Image: {previous_image_description}"
    payload = {
        "prompt": combined_description[:200],  # Max 200 characters for prompt
        "img_size": 1024,
        "guidance_scale": 7.5,
        "num_inference_steps": 50,
        "seed":  101,
    }
    response = requests.post(SDXL_URL, headers=headers, json=payload)
    if response.status_code == 200:
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"Image saved to {output_file}")
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Function to stitch images and text together into a final manga page
def stitch_images_and_text(images, text, output_file):
    print(images)
    width = max(img.width for img in images)
    height = sum(img.height for img in images)
    final_image = Image.new("RGB", (width, height), (255, 255, 255))

    y_offset = 0
    for i, img in enumerate(images):
        final_image.paste(img, (0, y_offset))
        y_offset += img.height
        text_position = (10, y_offset - 20)
        draw = ImageDraw.Draw(final_image)
        draw.text(text_position, text[i], fill=(0, 0, 0))

    final_image.save(output_file)
    print(f"Final manga saved to {output_file}")

# Main function to generate a manga
def create_manga(prompt, max_images= 6, max_tokens=500):
    # Step 1: Evaluate the Initial Prompt
    if not evaluate_prompt(prompt):
        print("Prompt is not detailed enough. Enhancing the prompt...")
        prompt = enhance_prompt(prompt)

    context = ""
    images = []
    story_text = []
    #previous_image_description = "An empty spaceship with two passengers in cryosleep."  # Initial description
    previous_image_description = "A gigantic palace full of monkeys and monkey king."  # Initial description

    for i in range(max_images):
        sentence = None

        print ("iteration to evaluate==================================================================" , i )

        #1st iteration relies on prompt alone
        if i == 0:
           sentence = generate_story_sentence_only_text_llava(prompt, context, None, max_tokens)
        else:
           sentence = generate_story_sentence_llava(summarize_story(prompt), summarize_story(context), f"scene_{i}.png", max_tokens)

        print ("sentence generated is for =>",i, sentence)
        if not sentence:  # If no sentence is generated, stop
            sentence = generate_story_sentence_only_text_llava(prompt, previous_image_description, None, max_tokens)
            #break

        # Print the sentence associated with each image
        print(f"Sentence {i + 1}: {sentence}")

        # Step 2: Generate Image for the sentence, including context from the previous image
        output_image_file = f"scene_{i + 1}.png"
        summary_text = summarize_story(sentence+" "+previous_image_description)
        #generate_sdxl_image(sentence, previous_image_description, output_image_file)
        generate_sdxl_image(summary_text, previous_image_description, output_image_file)
        img = Image.open(output_image_file)
        images.append(img)
        story_text.append(sentence)

        # Update context with the new sentence
        context = sentence

        # Update previous image description based on the generated image (for simplicity, this is a placeholder)
        previous_image_description = f"A futuristic scene depicting {sentence[:100]}"

    print ("==================================================================" )
    stitch_images_and_text(images, story_text, "final_manga.pdf")

# Example usage:
#prompt = "Two passengers aboard a spaceship traveling to a distant colony planet are awoken from hypersleep decades too early. As they navigate life in isolation, they form a bond and uncover a life-threatening secret that could jeopardize the entire mission."
prompt = "A kingdom of monkeys tries to build a city using the resources from the jungle. An elephant army from another elphant kingdom attacks the monkey kingdom."
create_manga(prompt)